In [1]:
import tensorflow as tf
from tensorflow.keras import layers, models
from tensorflow.keras.preprocessing.image import ImageDataGenerator

c:\Users\amiya\.conda\envs\DL-new\Lib\site-packages\h5py\__init__.py:36: UserWarning: h5py is running against HDF5 1.14.6 when it was built against 1.14.5, this may cause problems
  _warn(("h5py is running against HDF5 {0} when it was built against {1}, "


In [ ]:
img_size=(96, 96)
batch_size=32
epochs=25

In [ ]:
train_gen=ImageDataGenerator(
    rescale=1/255,
    rotation_range=10,
    zoom_range=0.1,
    horizontal_flip=True
).flow_from_directory(
    'train_classification',
    target_size=img_size,
    batch_size=batch_size,
    class_mode='categorical'
)

Found 447 images belonging to 7 classes.


In [ ]:
val_gen=ImageDataGenerator(rescale=1/255).flow_from_directory(
    'valid_classification',
    target_size=img_size,
    batch_size=batch_size,
    class_mode='categorical'
)

Found 127 images belonging to 7 classes.


In [ ]:
model=models.Sequential([
    layers.Conv2D(16, (3,3),activation='relu',input_shape=(*img_size, 3)),
    layers.MaxPooling2D(2,2),

    layers.Conv2D(32, (3,3),activation='relu'),
    layers.MaxPooling2D(2,2),

    layers.Conv2D(64, (3,3),activation='relu'),
    layers.MaxPooling2D(2,2),

    layers.Flatten(),
    layers.Dense(64,activation='relu'),
    layers.Dense(train_gen.num_classes,activation='softmax')
])

c:\Users\amiya\.conda\envs\DL-new\Lib\site-packages\keras\src\layers\convolutional\base_conv.py:113: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


In [ ]:
model.compile(optimizer='adam',loss='categorical_crossentropy',metrics=['accuracy'])
history=model.fit(train_gen,validation_data=val_gen,epochs=epochs)

c:\Users\amiya\.conda\envs\DL-new\Lib\site-packages\keras\src\trainers\data_adapters\py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


Epoch 1/25
14/14 ━━━━━━━━━━━━━━━━━━━━ 9s 507ms/step - accuracy: 0.3311 - loss: 1.8204 - val_accuracy: 0.3858 - val_loss: 1.7702
Epoch 2/25
14/14 ━━━━━━━━━━━━━━━━━━━━ 6s 438ms/step - accuracy: 0.4004 - loss: 1.6158 - val_accuracy: 0.4567 - val_loss: 1.5507
Epoch 3/25
14/14 ━━━━━━━━━━━━━━━━━━━━ 6s 444ms/step - accuracy: 0.4497 - loss: 1.4100 - val_accuracy: 0.4724 - val_loss: 1.4788
Epoch 4/25
14/14 ━━━━━━━━━━━━━━━━━━━━ 6s 437ms/step - accuracy: 0.4989 - loss: 1.2479 - val_accuracy: 0.4724 - val_loss: 1.4233
Epoch 5/25
14/14 ━━━━━━━━━━━━━━━━━━━━ 6s 448ms/step - accuracy: 0.5593 - loss: 1.1450 - val_accuracy: 0.4331 - val_loss: 1.7461
Epoch 6/25
14/14 ━━━━━━━━━━━━━━━━━━━━ 6s 449ms/step - accuracy: 0.6107 - loss: 1.0756 - val_accuracy: 0.4803 - val_loss: 1.4153
Epoch 7/25
14/14 ━━━━━━━━━━━━━━━━━━━━ 6s 451ms/step - accuracy: 0.5772 - loss: 0.9827 - val_accuracy: 0.4646 - val_loss: 1.5784
Epoch 8/25
14/14 ━━━━━━━━━━━━━━━━━━━━ 6s 445ms/step - accuracy: 0.6532 - loss: 0.8872 - val_accuracy: 0.

In [7]:
model.save("cnn_model.h5")

In [ ]:
converter=tf.lite.TFLiteConverter.from_keras_model(model)
tflite_model=converter.convert()
with open("model.tflite", "wb") as f:
    f.write(tflite_model)

INFO:tensorflow:Assets written to: C:\Users\amiya\AppData\Local\Temp\tmplwuws8t0\assets


INFO:tensorflow:Assets written to: C:\Users\amiya\AppData\Local\Temp\tmplwuws8t0\assets


Saved artifact at 'C:\Users\amiya\AppData\Local\Temp\tmplwuws8t0'. The following endpoints are available:

* Endpoint 'serve'
  args_0 (POSITIONAL_ONLY): TensorSpec(shape=(None, 96, 96, 3), dtype=tf.float32, name='keras_tensor')
Output Type:
  TensorSpec(shape=(None, 7), dtype=tf.float32, name=None)
Captures:
  2509683469072: TensorSpec(shape=(), dtype=tf.resource, name=None)
  2509683470224: TensorSpec(shape=(), dtype=tf.resource, name=None)
  2509683468880: TensorSpec(shape=(), dtype=tf.resource, name=None)
  2509683468496: TensorSpec(shape=(), dtype=tf.resource, name=None)
  2509683468304: TensorSpec(shape=(), dtype=tf.resource, name=None)
  2509683469648: TensorSpec(shape=(), dtype=tf.resource, name=None)
  2509683470608: TensorSpec(shape=(), dtype=tf.resource, name=None)
  2509683469456: TensorSpec(shape=(), dtype=tf.resource, name=None)
  2509683469840: TensorSpec(shape=(), dtype=tf.resource, name=None)
  2509683466576: TensorSpec(shape=(), dtype=tf.resource, name=None)


In [9]:
test_gen=ImageDataGenerator(rescale=1/255).flow_from_directory(
    'test_classification',
    target_size=(96,96),
    batch_size=1,
    class_mode='categorical',
    shuffle=False
)

loss,acc=model.evaluate(test_gen)
print(f"Test Accuracy:{acc*100}%")

Found 63 images belonging to 7 classes.
 7/63 ━━━━━━━━━━━━━━━━━━━━ 0s 9ms/step - accuracy: 0.4020 - loss: 0.6828     

c:\Users\amiya\.conda\envs\DL-new\Lib\site-packages\keras\src\trainers\data_adapters\py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


63/63 ━━━━━━━━━━━━━━━━━━━━ 1s 10ms/step - accuracy: 0.6508 - loss: 1.1311
Test Accuracy:65.07936716079712%
